# Logical Gate Benchmarking

## Imports

In [ ]:
import numpy as np

import matplotlib as mpl
mpl.rcParams["figure.dpi"] = 500

import matplotlib.pyplot as plt
plt.rcParams.update({
    "text.usetex": True,
    "font.size": 18,
    "font.family": "serif",
})

from qiskit import QuantumCircuit
from qiskit.circuit.library import XGate, YGate, ZGate, HGate, SGate, TGate, CXGate, CYGate, CZGate, RXGate, RYGate, RZGate
from qiskit.quantum_info import Statevector, state_fidelity

from LogicalQ.Logical import LogicalCircuit, LogicalStatevector, logical_state_fidelity
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.HardwareModels import hardware_models_Quantinuum
from LogicalQ.Execution import execute_circuits
from LogicalQ.Analysis import counts_to_statevector

## Non-parametric gates

In [ ]:
min_n_gates, max_n_gates, step_n_gates = 0, 4, 1
n_gates_data = np.pow(2, np.arange(min_n_gates, max_n_gates+1, step_n_gates))

In [ ]:
data = {}

gates = [
    # Single-qubit gates
    XGate,
    YGate,
    ZGate,
    HGate,
    SGate,
    TGate,
    # Multi-qubit gates (slow!)
    # CXGate,
    # CYGate,
    # CZGate,
]

for gate in gates:
    # 1. Construct circuits
    qc_list = []
    lqc_list = []
    sv_exact_list = []
    for n_gates in n_gates_data:
        # a. Construct physical circuit
        qc_gate = QuantumCircuit(gate().num_qubits)
        qc_gate.append(gate(), list(range(gate().num_qubits)))
    
        # b. Construct logical circuit
        lqc_gate = LogicalCircuit.from_physical_circuit(qc_gate, **steane_code)
        
        # c. Construct exact statevector
        sv_gate_exact = Statevector(qc_gate)

        qc_gate.measure_all()
        lqc_gate.measure_all()

        qc_list.append(qc_gate)
        lqc_list.append(lqc_gate)
        sv_exact_list.append(sv_gate_exact)

    # 2. Simulate circuits
    shots_physical = int(1E5)
    shots_logical = int(1E4)
    results_physical = execute_circuits(
        qc_list,
        backend="aer_simulator",
        method="statevector",
        hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None,
        shots=shots_physical,
    )
    results_logical = execute_circuits(
        lqc_list,
        backend="aer_simulator",
        method="statevector",
        hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None,
        shots=shots_logical,
    )

    # 3. Analyze results
    physical_data = []
    for sv_exact, result_physical in zip(sv_exact_list, results_physical):
        sv_physical = counts_to_statevector(result_physical.get_counts())
        fidelity_physical = state_fidelity(sv_exact, sv_physical)
        physical_data.append(fidelity_physical)
    logical_data = []
    for sv_exact, result_logical in zip(sv_exact_list, results_logical):
        sv_logical = LogicalStatevector.from_counts(result_logical.get_counts(), gate().num_qubits, **steane_code)
        fidelity_logical = logical_state_fidelity(sv_exact, sv_logical)
        logical_data.append(fidelity_logical)

    physical_data = np.round(physical_data, int(np.log10(shots_physical)+1))
    logical_data = np.round(logical_data, int(np.log10(shots_logical)+1))

    # n. Save data
    data[gate().name] = {
        "physical_data": physical_data,
        "logical_data": logical_data,
    }

data

In [ ]:
fig, ax = plt.subplots(ncols=len(gates), sharey="row", figsize=(40,8), dpi=128)

for g, gate in enumerate(gates):
    ax[g].scatter(n_gates_data, data[gate().name]["physical_data"], label="$\\mathcal{F}_P$")
    ax[g].scatter(n_gates_data, data[gate().name]["logical_data"], label="$\\mathcal{F}_L$")

    ax[g].set_title(f"$\\hat{{{gate().name.upper()}}}$")

plt.suptitle("Fidelity loss of non-parametric gates: Physical vs. Logical")
plt.legend()

plt.tight_layout()

plt.savefig("./data/logical_gate_benchmarking-nonparametric.png", dpi=128)
plt.show()